In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("train.csv")
submit_data = pd.read_csv("test.csv")

In [3]:
data_address = pd.DataFrame(np.zeros((submit_data.shape[0],6),dtype=int),columns=["Mr","Mrs","Miss","Master","Dr","Other"])

In [4]:
data = pd.concat([submit_data,data_address],axis=1)

In [5]:
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Mr,Mrs,Miss,Master,Dr,Other
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,0,0,0,0,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0,0,0,0,0,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,0,0,0,0,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,0,0,0,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0,0,0,0,0,0


In [6]:
# data_MrAge = data[0:0]
# data_MrsAge = data[0:0]
# data_MissAge = data[0:0]
# data_MasterAge = data[0:0]
# data_DrAge = data[0:0]
# data_OtherAge = data[0:0]

In [7]:
import math

def append_age(row,axis=0):    
#     global data_MrAge
#     global data_MrsAge
#     global data_MissAge
#     global data_MasterAge
#     global data_DrAge
#     global data_OtherAge
    
    i = int(row["PassengerId"])-1
    name = row["Name"]
    if "Mrs" in name:
#             data_MrsAge = pd.concat([data_MrsAge,data[i:i+1]],axis=0)
            row["Mrs"] = 1.0
    elif "Mr" in name:
#             data_MrAge = pd.concat([data_MrAge,data[i:i+1]],axis=0)
            row["Mr"] = 1.0
    elif "Miss" in name:
#             data_MissAge = pd.concat([data_MissAge,data[i:i+1]],axis=0)
            row["Miss"] = 1.0
    elif "Master" in name:
#             data_MasterAge = pd.concat([data_MasterAge,data[i:i+1]],axis=0)
            row["Master"] = 1.0
    elif "Dr" in name:
#             data_DrAge = pd.concat([data_DrAge,data[i:i+1]],axis=0)
            row["Dr"] = 1.0
    else:
#             data_OtherAge = pd.concat([data_OtherAge,data[i:i+1]],axis=0)
            row["Other"] = 1.0
    return row

In [8]:
def sep_age(data):
    data = data.apply(append_age,axis=1)
    return data
expanded_data = sep_age(data)

In [9]:
processed_data = expanded_data.drop(["Name","Ticket","Cabin","Fare"],axis=1)
processed_data = pd.get_dummies(processed_data)
processed_data.head()

,PassengerId,Pclass,Age,SibSp,Parch,Mr,Mrs,Miss,Master,Dr,Other,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0,1,0,1,0
1,893,3,47.0,1,0,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0,0,1
2,894,2,62.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0,1,0,1,0
3,895,3,27.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,1
4,896,3,22.0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0,0,1


In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
data_withAge = processed_data[pd.isna(processed_data["Age"]) == False]
data_withoutAge = processed_data[pd.isna(processed_data["Age"])]

In [12]:
LR = LinearRegression()
LR.fit(data_withAge[["Pclass","SibSp","Parch","Mr","Mrs","Miss","Master","Dr","Other","Sex_female","Sex_male","Embarked_C","Embarked_Q","Embarked_S"]],data_withAge["Age"])

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [13]:
data_withoutAge["Age"] = LR.predict(data_withoutAge[["Pclass","SibSp","Parch","Mr","Mrs","Miss","Master","Dr","Other","Sex_female","Sex_male","Embarked_C","Embarked_Q","Embarked_S"]])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
def round_age(row):
    age = row["Age"]
    rounded_age = round(age)
    row["Age"] = rounded_age
    return row

data_withoutAge = data_withoutAge.apply(round_age,axis=1)

In [15]:
data_ageFilled = pd.concat([data_withAge,data_withoutAge],axis=0)

In [16]:
data["Age"] = data_ageFilled["Age"]

In [17]:
data = sep_age(data)

In [18]:
data.to_csv("test_AgeFilled.csv")